In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))



DGL backend not selected or invalid.  Assuming PyTorch for now.
/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)

        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [11]:
def po_linear_model_sqrt_expo(graph, alpha=1, beta=1, sigma=0.1, gamma=1):    
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * np.sqrt(sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i])

In [ ]:
# cluster-level 
repeat_num = 1000

GATE_hat_list = np.zeros((repeat_num, len(ramps)))

for seed in tqdm(range(repeat_num)):    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_cluster,))   
    
    for num_step in range(len(ramps)): 
        print(f"Seed: {seed}")
        torch.manual_seed(2)
        p_list = ramps[num_step:]        
        g_feat_list = []
        y_list = []
        
        for p in p_list:            
            tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]            
            tr_units = reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters])  
                        
            nx.set_node_attributes(g, 0, "z")
            nx.set_node_attributes(g, {unit:1 for unit in tr_units}, "z")
            
            po_linear_model_sqrt_expo(g, gamma = 1)            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)        
    
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    torch.save(GATE_hat_list, "Result/SQRT_expo_CR_cluster.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.02, Loss: 7.9085
Epoch 0, Treat Prop: 0.05, Loss: 2.3309
Epoch 0, Treat Prop: 0.10, Loss: 1.4804
Epoch 0, Treat Prop: 0.25, Loss: 2.2351
Epoch 0, Treat Prop: 0.50, Loss: 4.2810
Epoch 50, Treat Prop: 0.02, Loss: 0.1917
Epoch 50, Treat Prop: 0.05, Loss: 0.2145
Epoch 50, Treat Prop: 0.10, Loss: 0.1527
Epoch 50, Treat Prop: 0.25, Loss: 0.3220
Epoch 50, Treat Prop: 0.50, Loss: 0.7069
Epoch 100, Treat Prop: 0.02, Loss: 0.0768
Epoch 100, Treat Prop: 0.05, Loss: 0.0383
Epoch 100, Treat Prop: 0.10, Loss: 0.0616
Epoch 100, Treat Prop: 0.25, Loss: 0.0924
Epoch 100, Treat Prop: 0.50, Loss: 0.0728
Epoch 150, Treat Prop: 0.02, Loss: 0.0629
Epoch 150, Treat Prop: 0.05, Loss: 0.0310
Epoch 150, Treat Prop: 0.10, Loss: 0.0516
Epoch 150, Treat Prop: 0.25, Loss: 0.0793
Epoch 150, Treat Prop: 0.50, Loss: 0.0605
Epoch 200, Treat Prop: 0.02, Loss: 0.0602
Epoch 200, Treat Prop: 0.05, Loss: 0.0303
Epoch 200, Treat Prop: 0.10, Loss: 0.0469
Epoch 200, Treat Prop: 0.25, Loss: 0.0761
Epoch 2

  0%|          | 1/1000 [00:35<9:51:47, 35.54s/it]

...

0.034578390419483185
Seed: 103
Epoch 0, Treat Prop: 0.02, Loss: 7.9391
Epoch 0, Treat Prop: 0.05, Loss: 2.2548
Epoch 0, Treat Prop: 0.10, Loss: 1.4485
Epoch 0, Treat Prop: 0.25, Loss: 2.4310
Epoch 0, Treat Prop: 0.50, Loss: 4.2928
Epoch 50, Treat Prop: 0.02, Loss: 0.2203
Epoch 50, Treat Prop: 0.05, Loss: 0.1980
Epoch 50, Treat Prop: 0.10, Loss: 0.1511
Epoch 50, Treat Prop: 0.25, Loss: 0.2779
Epoch 50, Treat Prop: 0.50, Loss: 0.5340
Epoch 100, Treat Prop: 0.02, Loss: 0.0698
Epoch 100, Treat Prop: 0.05, Loss: 0.0282
Epoch 100, Treat Prop: 0.10, Loss: 0.0593
Epoch 100, Treat Prop: 0.25, Loss: 0.0635
Epoch 100, Treat Prop: 0.50, Loss: 0.0623
Epoch 150, Treat Prop: 0.02, Loss: 0.0750
Epoch 150, Treat Prop: 0.05, Loss: 0.0265
Epoch 150, Treat Prop: 0.10, Loss: 0.0509
Epoch 150, Treat Prop: 0.25, Loss: 0.0676
Epoch 150, Treat Prop: 0.50, Loss: 0.0518
Epoch 200, Treat Prop: 0.02, Loss: 0.0704
Epoch 200, Treat Prop: 0.05, Loss: 0.0259
Epoch 200, Treat Prop: 0.10, Loss: 0.0465
Epoch 200, Treat P

 10%|█         | 104/1000 [59:58<8:27:52, 34.01s/it]

0.03359741345047951
Seed: 104
Epoch 0, Treat Prop: 0.02, Loss: 7.6095
Epoch 0, Treat Prop: 0.05, Loss: 2.2058
Epoch 0, Treat Prop: 0.10, Loss: 1.4284
Epoch 0, Treat Prop: 0.25, Loss: 3.1036
Epoch 0, Treat Prop: 0.50, Loss: 4.2814
Epoch 50, Treat Prop: 0.02, Loss: 0.1846
Epoch 50, Treat Prop: 0.05, Loss: 0.2293
Epoch 50, Treat Prop: 0.10, Loss: 0.1548
Epoch 50, Treat Prop: 0.25, Loss: 0.2980
Epoch 50, Treat Prop: 0.50, Loss: 0.4336
Epoch 100, Treat Prop: 0.02, Loss: 0.0306
Epoch 100, Treat Prop: 0.05, Loss: 0.0368
Epoch 100, Treat Prop: 0.10, Loss: 0.0479
Epoch 100, Treat Prop: 0.25, Loss: 0.0453
Epoch 100, Treat Prop: 0.50, Loss: 0.0745
Epoch 150, Treat Prop: 0.02, Loss: 0.0446
Epoch 150, Treat Prop: 0.05, Loss: 0.0298
Epoch 150, Treat Prop: 0.10, Loss: 0.0638
Epoch 150, Treat Prop: 0.25, Loss: 0.0531
Epoch 150, Treat Prop: 0.50, Loss: 0.0631
Epoch 200, Treat Prop: 0.02, Loss: 0.0860
Epoch 200, Treat Prop: 0.05, Loss: 0.0304
Epoch 200, Treat Prop: 0.10, Loss: 0.0597
Epoch 200, Treat Pr

 10%|█         | 105/1000 [1:00:32<8:26:31, 33.96s/it]

0.046856630593538284
Seed: 105
